In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import gensim
from gensim.models import word2vec
from collections import defaultdict
import nltk
import re
prog = re.compile('\.[A-Z]')
Head_check = re.compile('^[A-Z][^A-Z]+$')
from spacy.lang.en import English
nlp = English()
import chainer
from chainer import datasets
from chainer import functions as F
from chainer import links as L
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 50)

In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [3]:
len(train_df) + len(test_df)

1902194

In [4]:
all_document = pd.concat([train_df["comment_text"],test_df["comment_text"]])
all_document.shape

(1902194,)

In [18]:
%%time
t = []
for s in train_df['comment_text']:
    t.append(nltk.word_tokenize(s))

CPU times: user 9min 25s, sys: 1.86 s, total: 9min 27s
Wall time: 9min 27s


In [19]:
del_list = [",", ".", "?", "!", "'"]
for i in range(len(t)):
    for del_ele in del_list:
        if del_ele in t[i]:
            t[i].remove(del_ele)

In [20]:
words = {}
for i in range(len(t)):
    for word in t[i]:
        if word not in words:
            words[word] = len(words)

In [12]:
words

697248

In [13]:
t_vec = []
for i in range(len(t)):
    sentence_ids = []
    for word in t[i]:
        sentence_ids.append(words[word])
    sentence_ids = np.array(sentence_ids, dtype="float32")
    t_vec.append(sentence_ids)


In [14]:
type(t_vec[0])

numpy.ndarray

In [16]:
y = train_df['target'].values
len(y)

1804874

In [17]:
X_train, X_test, y_train, y_test = train_test_split(t_vec, y, train_size=0.6)

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
len(X_train) == int(np.floor(len(y)*0.6))

True

In [19]:
train =  datasets.tuple_dataset.TupleDataset(X_train, y_train)
test = datasets.tuple_dataset.TupleDataset(X_test, y_test)

In [32]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def word_compile(word, lemmatize = True):
    # 先頭のみ大文字
    check = Head_check.match(word)
    if check is not None:
        word = word[0].lower() + word[1:]
    # 見出し語化
    if lemmatize:
        word = lemmatizer.lemmatize(word)
    return word

NameError: name 'nl' is not defined

In [22]:
def splitter(text, split_words, include = True):
    split_words = [i.span()[0] for i in split_words]
    word_list = []
    word = ""
    for i in range(len(text)):
        word += text[i]
        if i in split_words:
            word_list.append(word_compile(word[:-1]))
            if include:
                word_list.append(".")
            word = ""
    word_list.append(word_compile(word))
    return word_list

splitter("borrow.Incoming", prog.finditer("borrow.Incoming"))

['borrow', '.', 'incoming']

In [23]:
splitter("i an to", prog.finditer("i an to"))

["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"]

In [8]:
%%time
sentences = [[word_compile(word) if len(prog.findall(word)) == 0 else splitter(word, prog.finditer(word))[0] 
              for word in nltk.word_tokenize(sentence)] for sentence in train_df['comment_text']]

CPU times: user 13min 57s, sys: 2.43 s, total: 14min
Wall time: 13min 59s


In [12]:
sentences[0]

['this',
 'is',
 'so',
 'cool',
 '.',
 'it',
 "'s",
 'like',
 ',',
 "'would",
 'you',
 'want',
 'your',
 'mother',
 'to',
 'read',
 'this',
 '?',
 '?',
 "'",
 'really',
 'great',
 'idea',
 ',',
 'well',
 'done',
 '!']

In [70]:
MIN_COUNT = 10

In [71]:
model = word2vec.Word2Vec(sentences, window=5, min_count=MIN_COUNT , iter=5, negative=5)

In [90]:
len(sentences)

1902194

In [131]:
a = model.wv[sentences[0]]
a.shape

(27, 100)

In [77]:
model.wv.most_similar('suck')

[('screw', 0.7846056818962097),
 ('soak', 0.7200046181678772),
 ('blow', 0.71183180809021),
 ('gobble', 0.6828003525733948),
 ('sucking', 0.6570413708686829),
 ('screwed', 0.6553224921226501),
 ('fucked', 0.6482899188995361),
 ('wake', 0.6468736529350281),
 ('lighten', 0.6461282968521118),
 ('snuff', 0.6439306735992432)]

In [78]:
model.save("jigsaw_word2vec")

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [81]:
model = word2vec.Word2Vec.load("jigsaw_word2vec")

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [160]:
model.wv[sentences[0]].shape

(27, 100)

In [4]:
class jigsaw_dataset(chainer.dataset.DatasetMixin):
    def __init__(self, train=True, min_count=10, train_size=0.6):
        train_df = pd.read_csv("../input/train.csv")
        self.X = train_df['comment_text']
        self.n_train = int(np.floor(len(self.X)*train_size))
        self.n_test = len(self.X) - self.n_train
        self.y = train_df['target'].values
        self.y = self.y.astype("float32")
        self.train = train
        self.train_size=train_size
    
    def __len__(self):
        if self.train:
            return self.n_train
        else:
            return self.n_test
    
    def get_example(self, i):
        
        t = []
        for s in self.X:
            t.append(nltk.word_tokenize(s))
        
        del_list = [",", ".", "?", "!", "'"]
        for i in range(len(t)):
            for del_ele in del_list:
                if del_ele in t[i]:
                    t[i].remove(del_ele)
        
        model = word2vec.Word2Vec.load("jigsaw_word2vec")
        
#         words = {}
#         for j in range(len(t)):
#             for word in t[j]:
#                 if word not in words:
#                     words[word] = len(words)
        
        t_vec = []
        for j in range(len(t)):
            t_ids = []
            for word in t[j]:
#                 t_ids.append(words[word])
                t_ids.append(model.wv[word])
            t_ids = np.array(t_ids, dtype="float32")
            t_vec.append(t_ids)
        
        X_train, X_test, y_train, y_test = train_test_split(t_vec, self.y, train_size=self.train_size, random_state=1)
        
        train = datasets.tuple_dataset.TupleDataset(X_train, y_train)
        test = datasets.tuple_dataset.TupleDataset(X_test, y_test)
        
        if self.train:
            return train[i]
        else:
            return test[i]
        

In [5]:
test = jigsaw_dataset()

In [6]:
test.get_example(1)

/home/ohara_lab/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


KeyError: "word 'This' not in vocabulary"

In [5]:
class RNN(chainer.Chain):
    def __init__(self, n_vocab=2000, n_units=100):
        super(RNN, self).__init__(
            embed = L.EmbedID(n_vocab, n_units),
            l1 = L.LSTM(n_units, n_units),
            l2 = L.LSTM(n_units, n_units),
            l3 = L.Linear(n_units, 1)
        )
    
    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def forward(self, x):
        h0 = self.embed(x)
        h1 = self.l1(F.dropout(h0))
        h2 = self.l2(F.dropout(h1))
        y = self.l3(F.dropout(h2))
        return y


model = RNN()
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

In [6]:
train = jigsaw_dataset()
test = jigsaw_dataset()
train_iter = chainer.iterators.SerialIterator(train, 4)
test_iter = chainer.iterators.SerialIterator(test, 4, repeat=False, shuffle=False)

In [7]:
train_1, val_2, test_3 = chainer.datasets.get_ptb_words()

In [17]:
train_1[4]

4